# Feature Engineering Avancé sur California Housing

Dans ce notebook, nous allons explorer différentes techniques de feature engineering sur le dataset California Housing.
Nous verrons notamment :
- L'analyse exploratoire des données
- La détection et le traitement des outliers
- La réduction de la skewness
- L'analyse des corrélations
- La réduction de dimension avec PCA

In [ ]:
# Imports nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## 1. Chargement et Exploration des Données

In [ ]:
# Chargement du dataset
data = fetch_california_housing(as_frame=True)
df = data.frame.copy()
df.head()

In [ ]:
# Analyse rapide des données
print("Information sur le dataset:")
df.info()
print("\nStatistiques descriptives:")
df.describe()

## 2. Analyse des Distributions

In [ ]:
# Visualisation des distributions par histogrammes
df.hist(bins=30, figsize=(15, 10))
plt.tight_layout()
plt.show()

## 3. Détection des Outliers avec IQR

In [ ]:
def detect_outliers_iqr(column):
    """
    Détecte les outliers en utilisant la méthode IQR
    """
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return (column < lower) | (column > upper)

In [ ]:
# Analyse des outliers pour chaque feature
outlier_summary = pd.DataFrame({col: detect_outliers_iqr(df[col]).sum() 
                             for col in df.select_dtypes(include=np.number)}.items(),
                             columns=['Feature', 'Outlier_Count'])
print("Résumé des outliers par feature:")
outlier_summary.sort_values(by='Outlier_Count', ascending=False)

## 4. Traitement de la Skewness

In [ ]:
# Identification des colonnes skewées
skewed_cols = df.skew().sort_values(ascending=False)
skewed_cols = skewed_cols[skewed_cols > 0.75]
print("Colonnes avec une forte asymétrie (skewness > 0.75):", skewed_cols.index.tolist())

In [ ]:
# Application de la transformation Yeo-Johnson
pt = PowerTransformer(method='yeo-johnson')
df[skewed_cols.index] = pt.fit_transform(df[skewed_cols.index])

## 5. Analyse des Corrélations

In [ ]:
# Matrice de corrélation
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matrice de corrélation")
plt.show()

## 6. Réduction de Dimension avec PCA

In [ ]:
# Standardisation et PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5)
plt.title("Projection PCA (2D)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid()
plt.show()

print("Variance expliquée par PC1 + PC2:", np.sum(pca.explained_variance_ratio_[:2]))

## 7. Modèle Simple (Avant/Après Feature Engineering)

In [ ]:
# Préparation des données
X = df.drop(columns='MedHouseVal')
y = df['MedHouseVal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement et évaluation du modèle
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Métriques d'évaluation:")
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R² Score:", r2_score(y_test, y_pred))